# Capstone project - Week 1
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem

In this project we will try to predict popularity of newly opened restaurant given its location.
Specifically, this report will be targeted on stakeholders, who want to open **McDonald's** restaurant in **Kiev, Ukraine** and want to see if their selected location will be popular enough.
The project is focused on **chain fast food restaurant** for few reasons:
* Fast food restaurant is valued for possibility to hop in and out to grab some food on the commute, it is not popular because of high class cuisine.
* Chain restaurants look similar and have the same menu within given country. 
* Because of similarity, we can assume that popularity metric for each restaurant in chain is more dependant on it's location, than on its cuisine or interior.
* There are already quite a few opened **McDonald's** restaurants in **Kiev**, so we can have enough data to make predictions 

Speaking of location, fast food restaurant should be more popular if it is located near some significant point(s) of interest, such as shopping center, metro station, city center or train station. But how much each type of interest affect on the popularity? Is the selected place for the new restaurant is good enough, if it was chosen by simple criterias (e.g. near metro station)? These are the questions I am trying to answer with this project.


## Data and how it will be used to solve the problem

Based on the Business Problem, we will use multiple metrics from the **Foursquare API**:
* Number of McDonald's restaurants in Kiev
* Location for each restaurant
* Popularity of each restaurant (amount of visitors)

Unfortunately, third metric is not that simple to recover from the API. For this metric we could use total amount of check-ins for each restaurant. After some initial research, with the current version of **Foursquare API** we cannot retrive amount of check-ins anymore.<br>
Other two possible resolutions for this metric are number of *'Likes'* for each restaurant and number of *'Rating signals'*.

I've decided to stick with the amount of *'Rating Signals'* as during the initial research it seems that there are more *'Rating Signals'* than there are *'Likes'* per restaurant. Therefore results should be more precise.<br>
Rating for each restaurant is a value between 1 and 10. *Rating Signals* is the total amount of people, who rated this venue. As we are not interested in the rating itself, we will use only amount of votes which will be our indicator of popularity for the venue. More people visited place means more people rated it.<br>
But what if one restaurant was opened two years ago and have only 100 *Rating Signals* while another one is opened for 10 years and has 1000 *Rating Signals*? Can we assume that second restaurant is more popular than the first one only by the amount of *Rating Signals*? No. To solve this issue we will also use one more metric for each restaurant:
* Venue Creation Date


This is a date, when the restaurant was added to **Forsquare**. From this value we will calculate average amount of *Rating Signals* per year. This value will be our main metric for the venue popularity.
After that we will also normalize these values, so that we will have values from 0 to 1. Let's call this normalized value as **Venue Popularity Index**, or **VPI**.
The closer **VPI** is to 1, the higher is the popularity.

After calculating **VPI** for each McDonald's restaurant in Kiev, we can plot these values on the map of the city  and predict **VPI** for future restaurants, based on their location.

